In [1]:
from incremental_td import incremental_TD
import numpy as np
import time

In [15]:
nrep = 1

# gen data from mech1
m, T, r = 200, 100, 10
sigma = 0.1
np.random.seed(5)
Ut, R = np.linalg.qr(np.random.normal(size = (m, r)))
Vt, R = np.linalg.qr(np.random.normal(size = (m, r)))
M_mean = Ut.dot(np.diag([r-i for i in range(r)])).dot(Vt.T)
Tensor = np.zeros((m, m, T))
for i in range(T):
	Tensor[:,:,i] = M_mean + np.random.normal(scale = sigma, size = (m, m))

# prepare starting points for DTA and TD
# reset r here
r = 10
Ut = Ut[:,:r]
Vt = Vt[:,:r]
Us, d, Vs = np.linalg.svd(Tensor[:,:,1])
Us = Us[:,:r]
Vs = Vs[:,:r]
ds = d[:r]

In [8]:
#####################################################################
###   ITD                                                         ###
###   ITD                                                         ###
###   ITD                                                         ###
#####################################################################
nrep = 1

time_record = np.zeros(nrep)
mean_acc = np.zeros(nrep)

for rep in range(nrep):
	# holder for accuracy by T
	acc = np.zeros(T)

	# regenerate data
	for i in range(T):
		Tensor[:,:,i] = M_mean + np.random.normal(scale = sigma, size = (m, m))
	
	# set starting point
	U, V = Us, Vs
	s1 = s2 = np.ones(r)

	start_time = time.time()
	for i in range(T):
		s1, U, s2, V = incremental_TD(s1, U, s2, V, Tensor[:,:,i], i+1, tol=1e-7)
		M_hat = np.linalg.multi_dot([U, U.T, Tensor[:,:,i], V, V.T])
		acc[i] = np.linalg.norm(Tensor[:,:,i] - M_hat) ** 2 / np.linalg.norm(Tensor[:,:,i]) ** 2
	time_record[rep] = time.time() - start_time #;print(acc)
	mean_acc[rep] = acc.mean()


In [10]:
print(acc)
print(mean_acc.std())

print(time_record.mean())
print(time_record.std())

[ 0.69936795  0.50325847  0.4913502   0.50342984  0.50233414  0.505352
  0.50335998  0.50598826  0.50852189  0.51017835  0.50258921  0.50404849
  0.50230594  0.50623274  0.50435313  0.50392254  0.50288959  0.50500037
  0.5053564   0.50340031  0.50293172  0.50278229  0.51073294  0.50206933
  0.50262751  0.50842513  0.50331473  0.51009634  0.50809052  0.50703673
  0.50359247  0.50854313  0.5100977   0.50147077  0.50683724  0.50447939
  0.50226666  0.50792513  0.50696962  0.50964559  0.50809228  0.50870761
  0.50655829  0.51295161  0.50740369  0.50738774  0.50785632  0.50774695
  0.50587541  0.5095969   0.51166991  0.50948167  0.50891791  0.50661724
  0.50412783  0.50507748  0.51564211  0.51005658  0.51404536  0.50259154
  0.50843529  0.50775321  0.50712452  0.51087827  0.51120048  0.50085138
  0.50964952  0.50908163  0.50165369  0.50865924  0.508585    0.51308735
  0.51148201  0.5082118   0.51032795  0.50313825  0.5134144   0.51018322
  0.5149169   0.51151725  0.51159762  0.50903859  0.5

In [3]:
def dtavec(U, s, x, f = 1):
	n = x.shape[1]
	r = U.shape[1]
	for i in range(n):
		xx = x[:, i]
		for j in range(r):
			y = np.dot(U[:, j], xx)
			s[j] = f * s[j] + y ** 2
			e = xx - y * U[:, j]
			U[:, j] += y / s[j] * e
			xx -= y * U[:, j]
			U[:, j] = U[:, j] / np.linalg.norm(U[:, j])
		U_new, R = np.linalg.qr(U)
		if not np.allclose(U_new[:,0], U[:, 0]):
			U_new[:,0] = -U_new[:,0]
	return U_new, s

def dta(s, U, s2, V, x, f = 1):
	U_new, s = dtavec(U, s, x, f)
	V_new, s2 = dtavec(V, s2, x.T, f)
	return s, U_new, s2, V_new

In [16]:
nrep = 1
#####################################################################
###   DTA                                                         ###
###   DTA                                                         ###
###   DTA                                                         ###
#####################################################################

time_record = np.zeros(nrep)
mean_acc = np.zeros(nrep)

for rep in range(nrep):
	# holder for accuracy by T
	acc = np.zeros(T)

	# regenerate data
	for i in range(T):
		Tensor[:,:,i] = M_mean + np.random.normal(scale = sigma, size = (m, m))
	
	# set starting point
	U, V = Us, Vs
	s1 = s2 = ds ** 2

	start_time = time.time()
	for i in range(T):
		s1, U, s2, V = dta(s1, U, s2, V, Tensor[:,:,i], f = 1)
		M_hat = np.linalg.multi_dot([U, U.T, Tensor[:,:,i], V, V.T])
		acc[i] = np.linalg.norm(Tensor[:,:,i] - M_hat) ** 2 / np.linalg.norm(Tensor[:,:,i]) ** 2
	time_record[rep] = time.time() - start_time
	mean_acc[rep] = acc.mean()

In [18]:
s1

array([ 10440.42743415,   8570.72038097,   6839.12367592,   5327.5349082 ,
         4003.89726115,   2893.1940589 ,   2000.93734118,   1297.1348857 ,
          785.16602024,    411.56431063])

In [17]:
print(acc)
print(mean_acc.std())

print(time_record.mean())
print(time_record.std())


[ 0.99999532  0.99999802  0.99999884  0.99999938  0.99999951  0.99999947
  0.9999997   0.99999969  0.99999982  0.99999982  0.99999987  0.99999984
  0.99999984  0.99999992  0.99999986  0.99999989  0.99999993  0.99999991
  0.99999992  0.99999989  0.99999994  0.99999995  0.99999998  0.99999998
  0.99999997  0.99999997  0.99999998  0.99999998  0.99999997  0.99999999
  0.99999999  0.99999998  0.99999997  0.99999998  0.99999998  0.99999998
  0.99999998  0.99999998  0.99999999  0.99999999  0.99999999  0.99999999
  0.99999999  0.99999999  0.99999999  0.99999999  0.99999999  0.99999999
  1.          0.99999999  0.99999999  0.99999999  0.99999999  0.99999999
  0.99999999  0.99999999  0.99999999  0.99999999  0.99999999  0.99999999
  1.          0.99999999  1.          1.          0.99999999  1.
  0.99999999  1.          0.99999999  1.          1.          1.          1.
  0.99999999  0.99999999  1.          1.          1.          1.          1.
  1.          1.          1.          1.          0

In [26]:
#####################################################################
###   GTRUTH                                                      ###
###   GTRUTH                                                      ###
###   GTRUTH                                                      ###
#####################################################################
nrep = 20
time_record = np.zeros(nrep)
mean_acc = np.zeros(nrep)

for rep in range(nrep):
	# holder for accuracy by T
	acc = np.zeros(T)

	# regenerate data
	for i in range(T):
		Tensor[:,:,i] = M_mean + np.random.normal(scale = sigma, size = (m, m))

	U, V = Ut, Vt
	start_time = time.time()
	for i in range(T):
		M_hat = np.linalg.multi_dot([U, U.T, Tensor[:,:,i], V, V.T])
		acc[i] = np.linalg.norm(Tensor[:,:,i] - M_hat) ** 2 / np.linalg.norm(Tensor[:,:,i]) ** 2
	time_record[rep] = time.time() - start_time
	mean_acc[rep] = acc.mean()

In [27]:
print(mean_acc.mean())
print(mean_acc.std())

0.508225116946
0.000283430264528


In [14]:
d

array([  1.01843216e+01,   9.27670481e+00,   8.12846074e+00,
         7.25078442e+00,   6.41574150e+00,   5.42042017e+00,
         4.41428781e+00,   3.56238208e+00,   3.02585738e+00,
         2.74119820e+00,   2.70763365e+00,   2.63362992e+00,
         2.63093839e+00,   2.59349080e+00,   2.55871410e+00,
         2.53525613e+00,   2.50725236e+00,   2.47241555e+00,
         2.46310868e+00,   2.43834332e+00,   2.42062146e+00,
         2.40441769e+00,   2.36081894e+00,   2.33347741e+00,
         2.29403715e+00,   2.26316166e+00,   2.24592359e+00,
         2.24079584e+00,   2.20798326e+00,   2.20109801e+00,
         2.19127568e+00,   2.16093334e+00,   2.15516943e+00,
         2.13807881e+00,   2.10909539e+00,   2.08573779e+00,
         2.06292162e+00,   2.05582874e+00,   2.02989924e+00,
         2.01349716e+00,   1.97597587e+00,   1.96362823e+00,
         1.94441758e+00,   1.94029435e+00,   1.91905373e+00,
         1.90983568e+00,   1.90094816e+00,   1.89094268e+00,
         1.87527336e+00,